In [2]:
import pandas as pd
import requests
import io

In [17]:
state_info = pd.read_csv('./source_data/state_info.csv', dtype = 'object')
dfs = []

for state in state_info['state_po']:
    r = requests.get(f'https://fred.stlouisfed.org/graph/fredgraph.csv?id={state}RGSP')
    df = pd.read_csv(io.BytesIO(r.content))
    df['year'] = pd.to_datetime(df['DATE']).dt.year
    df['gdp'] = df[f'{state}RGSP']
    df['state'] = state
    df = df[['year', 'state', 'gdp']]
    df = df.sort_values(by = 'year', ascending = True)

    df_1 = df.groupby('state')[['year', 'gdp']]\
            .apply(lambda table : table.set_index('year').pct_change())\
            .reset_index()\
            .rename(columns = {'gdp':'gdp_1'})

    df_4 = df.groupby('state')[['year', 'gdp']]\
            .apply(lambda table : table.set_index('year').pct_change(4))\
            .reset_index()\
            .rename(columns = {'gdp':'gdp_4'})

    df = pd.merge(left = df_1, right = df_4, how = 'inner', on = 'year')
    df = df[['year', 'state_x', 'gdp_1', 'gdp_4']]\
                    .rename(columns = {'state_x':'state'})

    dfs.append(df)

all_data = pd.concat(dfs, axis = 0, ignore_index = True)
all_data.to_csv('./processed_data/1997-2022-gdp.csv', index = False, header = True)